# Spotify Mood Based Song Retreiver

## 1. Getting and Cleaning the Data

In [1]:
import requests as rq
import bs4

In [6]:
# Extract data from the link
def data_extractor(link):
    print("Retrieving body text and title text from:", link)
    res = rq.get(link)
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    body = soup.find('body').text
    title = soup.find('title').text
    body = body.replace('\n', ' ')[:4000] #Change this to change the amount of text extracted
    return body, title

# 2. Defining the API Key for the Model

In [28]:
# Get the API key from the .env file
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "True"

# 3. Using the LLM to Obtain a Song based on the User Text

In [54]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.docstore.document import Document

# Summarize the text
def song_retrieve(link):
    # Define a prompt
    prompt_template = "You are a song recommendations system. Based on the following text including the title and body of the article, understand the mood of the reader and generate exactly 1 song recommendation. Do not output any text apart from the name of the song and the artist. \n\nTitle: {title} \n\nBody: {body}"
    prompt = PromptTemplate(input_variables=["title", "body"], template=prompt_template)

    # Load the chain
    llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro")
    chain = prompt | llm | StrOutputParser()

    # Extract data from the link
    body, title = data_extractor(link)

    # Run the chain
    result = chain.invoke({"title": title, "body": body})

    return result

In [56]:
song_retrieve(
    "https://www.reddit.com/r/depression/comments/ogvn4h/im_just_sad_im_just_so_fucking_sad/"
)

Retrieving body text and title text from: https://www.reddit.com/r/depression/comments/ogvn4h/im_just_sad_im_just_so_fucking_sad/


'Sadness and Sorrow - Pink Floyd'